# Advent of Code
## Day 8: 
### Part One



* acc increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
* jmp jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
* nop stands for No OPeration - it does nothing. The instruction immediately below it is executed next.

---

In [1]:
# Read in the inputs
# Read them in as an (operation, value) pair
inputs = [tuple(i[:-1].split(' ')) for i in open("Day08_input.txt").readlines()]

# Clean the inputs so that each value is an integer
for i, (instruction, val) in enumerate(inputs):
    inputs[i] = (instruction, int(val))

In [2]:
lines_executed = {len(inputs)}
acc = 0
line_counter = 0

while line_counter not in lines_executed:
    lines_executed |= {line_counter}
    instruction, val = inputs[line_counter]
    
    if instruction == 'acc':
        acc += val
        line_counter += 1
    elif instruction == 'jmp':
        line_counter += val
    elif instruction == 'nop':
        line_counter += 1
    
    

print(f"Program will repeat on line {line_counter} with accumulator value {acc}")

Program will repeat on line 178 with accumulator value 1614


---

### Part Two

---

In [8]:
class InfiniteLoop(Exception):
    pass

def test_run(script):
        
    lines_executed = set()
    acc = 0
    line_counter = 0

    while (line_counter not in lines_executed) and (line_counter < len(script)):
        lines_executed |= {line_counter}
        instruction, val = script[line_counter]

        if instruction == 'acc':
            acc += val
            line_counter += 1
        elif instruction == 'jmp':
            line_counter += val
        elif instruction == 'nop':
            line_counter += 1
    
    if line_counter >= len(script):
        return acc
    else:
        raise InfiniteLoop("This script contains an infinite loop")

        
acc = None
changed_line = None
original_input = None
modified_input = None

for i, line in enumerate(inputs):
    
    modified_inputs = inputs.copy()
    instruction, val = line
    
    
    if instruction == 'jmp':
        modified_inputs[i] = ('nop', val)
        try:
            acc = test_run(modified_inputs)
        except InfiniteLoop:
            pass
    elif instruction == 'nop':
        modified_inputs[i] = ('jmp', val)
        try:
            acc = test_run(modified_inputs)
        except InfiniteLoop:
            pass
    else:
        pass
    
    if acc is not None:
        changed_line = i
        original_input = (instruction,val)
        modified_input = modified_inputs[changed_line]
        break
    
print(
    f"The program required a change to line {changed_line}\n - changing {original_input} to {modified_input} - \n"
    f"which allowed the program to terminate with accumulator value {acc}."
)

The program required a change to line 247
 - changing ('jmp', 209) to ('nop', 209) - 
which allowed the program to terminate with accumulator value 1260.


---